# Group Project 4 - Team T.W. Schultz


### Team Members: Neha Lala (*Team Manager*), Gavin Armstrong, Dojun Kim, Samantha Wang, Daniela Salinas Gomez

## **1.3 [A] Unit Test:**
## **1.6 [A] Construct Nutrient System:**
## **1.7 [A] Nutritional challenges & Policy Goal:**
## **1.8 [B] Policy Options**
## **1.9 [C] Policy Cost**
## **1.10 [C] Value of Technical Innovation.**
## **1.12 [A] Replication & Discussion**


## **Project Topic & Goal:**

Our project focuses on analyzing the nutrition breakdown in Guatemala by region (rural or urban) and, household size. We dive deep into demand for food within the groups broken down by the specified criteria and are focused on evaluating what kinds of policies might be effective at improving outcomes for particular populations with specific regards to nutritional staples. Typically, the food people choose to eat may be quite different from the foods that people should eat, from a nutritional perspective and our policies aim at closing the bag between the current nutritional consumption levels and the recommended levels of nutrient intake.

## **1.1 [A] Replicability & Documentation:**
### *Importing Packages*:


In [1]:
!pip install python-gnupg
!pip install eep153_tools
!pip install CFEDemands --upgrade --pre
!pip install xarray --upgrade
!pip install -r requirements.txt
!pip install eep153_tools --upgrade
!pip install ConsumerDemands

import cfe
from eep153_tools.sheets import read_sheets
import numpy as np
import pandas as pd
from cfe.estimation import drop_columns_wo_covariance
from cfe import Regression
import matplotlib.pyplot as plt

  Using cached python_gnupg-0.5.2-py2.py3-none-any.whl (20 kB)
  Using cached eep153_tools-0.12.4-py2.py3-none-any.whl (4.9 kB)
  Using cached CFEDemands-0.6.1-py2.py3-none-any.whl (45 kB)
  Using cached ray-2.11.0-cp39-cp39-manylinux2014_x86_64.whl (65.3 MB)
  Using cached xarray-2024.3.0-py3-none-any.whl (1.1 MB)
  Using cached dvc-3.50.0-py3-none-any.whl (451 kB)
  Using cached ConsumerDemands-0.4.2.dev0-py2.py3-none-any.whl (12 kB)
  Using cached platformdirs-3.11.0-py3-none-any.whl (17 kB)
  Using cached zc.lockfile-3.0.post1-py3-none-any.whl (9.8 kB)
  Using cached shtab-1.7.1-py3-none-any.whl (14 kB)
  Using cached dvc_task-0.4.0-py3-none-any.whl (21 kB)
  Using cached shortuuid-1.0.13-py3-none-any.whl (10 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl (31 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
  Using cached grandalf-0.8-py3-none-any.whl (41 kB)
  Using cached dvc_data-3.15.1-py3-none-any.whl (71 kB

## **1.4 [A] Choice of a dataset:**

In [2]:
url = 'https://docs.google.com/spreadsheets/d/1YupX14TogoubSrlHtG9OgfBdKUhsss29L63mcNDNCUo/'

guatemala_prices = read_sheets(url,sheet='Food Prices',nheaders=2)
guatemala_fct = read_sheets(url,sheet='FCT')
guatemala_hhc = read_sheets(url,sheet='Household Characteristics')
guatemala_expenditures = read_sheets(url,sheet='Food Expenditures')
price = guatemala_prices
fct = guatemala_fct
hhc = guatemala_hhc
expen = guatemala_expenditures



In [3]:
def clean_multiindex_df(df):
    # If the columns are a MultiIndex, join them with an underscore, otherwise just strip them
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(map(str, col)).strip() for col in df.columns.values]
    else:
        df.columns = [col.strip() for col in df.columns.tolist()]

    # Remove duplicated columns
    df = df.loc[:, ~df.columns.duplicated(keep='first')]

    # Drop columns that start with 'Unnamed'
    df = df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)

    # Remove duplicated rows
    df = df.loc[~df.index.duplicated(), :]

    return df

# Clean the DataFrames with MultiIndex columns
price_cleaned = clean_multiindex_df(price)
fct_cleaned = clean_multiindex_df(fct)
hhc_cleaned = clean_multiindex_df(hhc)
expen_cleaned = clean_multiindex_df(expen)

In [4]:
import numpy as np
if 'm' not in expen.columns:
    expen_cleaned['m'] = 1

expen = expen_cleaned.set_index(['i','t','m'])
expen.columns.name = 'j'

expen = expen.apply(lambda x: pd.to_numeric(x,errors='coerce'))
expen = expen.replace(0,np.nan)

hhc = hhc_cleaned
if 'm' not in hhc.columns:
    hhc['m'] = 1

hhc = hhc.set_index(['i','t','m'])
hhc.columns.name = 'j'

hhc = hhc.apply(lambda x: pd.to_numeric(x,errors='coerce'))

q = price_cleaned
if 'm' not in q.columns:
    q['m'] = 1

q = q.apply(lambda x: pd.to_numeric(x,errors='coerce'))
q = q.replace(0,np.nan)

fct.index.name = 'n'

fct = fct.apply(lambda x: pd.to_numeric(x,errors='coerce'))


In [5]:
expen.head()

,,j,Cooking Oil,Water (bottle),Avocado,Mineral Water,Garlic,Mushrooms,Celery,Rice,Peas,Corn Atole,...,Tea,Tomatoes,Tortillas,Tostadas,Beef Entrails,Chicken Giblets,Yogurt,Yucca,Carrots,Leafcutter Ants and Other Insect
i,t,m,,,,,,,,,,,,,,,,,,,,,
1,2000,Metropolitana,7.5,20.0,5.0,6.00,1.00,NaN,2.0,5.0,NaN,NaN,...,7.0,9.0,60.0,NaN,NaN,21.0,NaN,NaN,3.6,NaN
10,2000,Metropolitana,15.0,15.0,30.0,NaN,2.00,20.0,2.0,10.0,NaN,NaN,...,8.5,14.0,30.0,NaN,30.0,12.5,NaN,NaN,7.5,NaN
100,2000,Metropolitana,13.0,10.5,NaN,16.00,0.25,NaN,2.0,6.5,NaN,2.0,...,NaN,2.5,4.0,NaN,16.0,NaN,NaN,NaN,3.0,NaN
1000,2000,Central,9.5,10.0,NaN,NaN,1.50,NaN,1.5,10.0,NaN,NaN,...,NaN,9.0,60.0,NaN,NaN,NaN,NaN,NaN,6.0,NaN
1001,2000,Central,10.0,20.0,5.0,7.75,1.00,NaN,2.0,5.0,2.0,4.0,...,22.0,6.0,37.5,NaN,12.0,NaN,NaN,1.0,3.5,NaN


In [6]:
hhc.head()

,,j,Males 00-03,Females 00-03,Males 04-08,Females 04-08,Males 09-13,Females 09-13,Males 14-18,Females 14-18,Males 19-30,Females 19-30,Males 31-50,Females 31-50,Males 51-99,Females 51-99,log HSize,Rural
i,t,m,,,,,,,,,,,,,,,,
1,2000,Metropolitana,0,1,0,0,0,0,0,0,1,1,0,1,0,0,1.386294,0
2,2000,Metropolitana,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1.098612,0
3,2000,Metropolitana,0,0,0,0,2,0,0,0,1,0,1,1,1,0,1.791759,0
4,2000,Metropolitana,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0
5,2000,Metropolitana,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1.098612,0


In [7]:
q.head()

,,2000_Central,2000_Metropolitana,2000_Noroccidente,2000_Nororiente,2000_Norte,2000_Peten,2000_Suroccidente,2000_Suroriente,m
j,u,,,,,,,,,
Cooking Oil,lbs,5.010737,5.368647,4.545454,4.563468,5.368647,5.000000,3.863636,5.368647,1
Water (bottle),lbs,0.454545,0.240500,0.240500,0.264550,0.240500,0.300625,0.245700,0.288600,1
Avocado,lbs,1.865672,3.544776,1.492537,3.731343,1.492537,1.492537,1.492537,2.985075,1
Mineral Water,lbs,2.750779,2.731270,3.209243,3.209243,3.209243,3.851091,3.209243,3.209243,1
Garlic,lbs,4.545455,4.545455,4.545455,9.090909,3.787879,9.090909,4.545455,4.545455,1


In [8]:
fct.head()

,j,2000,Code,Nutrient,Agua,Energía,Proteina,Grasa Total,Carbo-hidratos,Fibra Diet. total,...,Colesterol,Potasio,Sodio,Zinc,Magnesio,Vit. B6,Vit. B12,Ac. Fólico,Folato Equiv. FD,Fracción Comestible
n,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,16029,NaN,0.00,884,0.00,100.00,0.00,0.0,...,0,0,0,0.00,0,0.00,0.00,0,0,1.00
1,NaN,NaN,3046,NaN,69.80,168,17.89,9.21,2.13,0.0,...,240,226,77,3.01,18,0.52,10.83,0,350,1.00
2,NaN,NaN,11005,NaN,73.23,160,2.00,14.66,8.53,6.7,...,0,485,7,0.64,29,0.26,0.00,0,81,0.74
3,NaN,NaN,17016,NaN,87.60,48,0.00,0.00,12.30,0.0,...,0,2,12,0.10,1,0.00,0.00,0,0,1.00
4,NaN,NaN,11006,NaN,63.80,134,5.30,0.20,29.30,0.0,...,0,529,19,1.00,0,0.00,0.00,3,0,0.94


## **1.5 [A] Estimate Demand System:**

In [9]:
import cfe

result = cfe.Regression(y=np.log(expen.stack()),d=hhc)

result.get_beta().sort_values(ascending=False)
result.to_pickle('./foo.pickle')
result = cfe.read_pickle('foo.pickle')
result.predicted_expenditures()

i     t     m              j             
1     2000  Metropolitana  Avocado             7.131134
                           Bananas            16.368000
                           Beans               9.351316
                           Beef              102.055889
                           Beef Entrails      13.977229
                                                ...    
7276  2000  Suroriente     Tostadas            8.796111
                           Water (bottle)      9.580927
                           Watermelon          7.935121
                           Wheat Flour         4.943983
                           Yucca               3.456677
Length: 515736, dtype: float64